# 프로젝트 4 : CNN으로 패션 아이템 구분

- 일반적 신경망은 다양한 형태의 입력에 대한 확장성이 떨어짐
- ex. 신발이 가운데 있는 것으로 학습하다 신발이 한쪽으로 치우치면 성능 저하
- 특징 추출 가중치 가운데 집중하도록 학습되었기 때문

### CNN

- 사물이 조금만 치우쳐도 인식하지 못하던 인공 신경망 문제를 이미지 전체를 필터링 하는 방식으로 해결

- 이미지를 추출하는 필터 학습, 필터 하나가 작은 신경망이 됨
- 구성 : 컨볼루션 계층, 풀링 계층, 최종 분류용 일반 신경망

- 컨볼루션 : 특징 추출 위해 단계별 다양한 필터를 적용하고, 필터 적용시마다 이미지 왼쪽 위에서 오른쪽 아래까지 밀면서 곱하고 더하는 작업
- 컨볼루션 필터 = 커널 : 이미지에 겹치는 작은 필터 3X# 또는 5X5 사용
- 풀링계층 : 필터를 거친 여러 특징 중 가장 중요한 특징 하나 골라냄(평균이나 최대값 사용)
- 스트라이드 : 필터를 몇칸씩 건너뛰며 적용할 지를 조절 하는 값
- 특징 맵 : 컨볼루션 거쳐 나온 새로운 이미지

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [3]:
EPOCHS = 40
BATCH_SIZE = 64

# 데이터셋 불러오기

In [4]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./.data',
                      train=True,
                      download=True,
                      transform=transforms.Compose([
                          transforms.RandomHorizontalFlip(),
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))
                          
                      ])),
        batch_size=BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./.data',
                      train=False,
                      transform=transforms.Compose([
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))
                      ])),
        batch_size=BATCH_SIZE, shuffle=True)

# cnn 신경망 설계

In [5]:
class Net(nn.Module) :
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  # 채널수 1(흑백), 10개
                                            # 특징맵, 커널 크기 5X5, 숫자하나 정사각형 간주
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    # 풀링 계층    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# 학습하기

In [7]:
def train(model, train_loader, optimizer, epoch) :
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader) :
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{}  ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
                100.*batch_idx / len(train_loader), loss.item()))

# 테스트하기

In [8]:
def evaluate(model, test_loader) :
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader :
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            # 배치 오차를 생산
            test_loss += F.cross_entropy(output, target,
                                        reduction = 'sum').item()
            
            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

# 실행

- 앞 프로젝트와 정확도 비교

In [9]:
for epoch in range(1, EPOCHS + 1) :
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
           epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000  (0%)]	Loss: 2.316802
Train Epoch: 1 [12800/60000  (21%)]	Loss: 1.445771
Train Epoch: 1 [25600/60000  (43%)]	Loss: 1.292153
Train Epoch: 1 [38400/60000  (64%)]	Loss: 0.919689
Train Epoch: 1 [51200/60000  (85%)]	Loss: 0.874145
[1] Test Loss: 0.4472, Accuracy: 86.20%
Train Epoch: 2 [0/60000  (0%)]	Loss: 0.682692
Train Epoch: 2 [12800/60000  (21%)]	Loss: 0.823187
Train Epoch: 2 [25600/60000  (43%)]	Loss: 0.584783
Train Epoch: 2 [38400/60000  (64%)]	Loss: 0.457481
Train Epoch: 2 [51200/60000  (85%)]	Loss: 0.399676
[2] Test Loss: 0.3197, Accuracy: 90.05%
Train Epoch: 3 [0/60000  (0%)]	Loss: 0.591260
Train Epoch: 3 [12800/60000  (21%)]	Loss: 0.664857
Train Epoch: 3 [25600/60000  (43%)]	Loss: 0.456080
Train Epoch: 3 [38400/60000  (64%)]	Loss: 0.479529
Train Epoch: 3 [51200/60000  (85%)]	Loss: 0.641049
[3] Test Loss: 0.2331, Accuracy: 93.47%
Train Epoch: 4 [0/60000  (0%)]	Loss: 0.304638
Train Epoch: 4 [12800/60000  (21%)]	Loss: 0.620368
Train Epoch: 4 [25600/60000  (43%